In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 22480



### data

In [2]:
sas.submitLST(f"""
DATA orders;
    INPUT Coffee $ Window $ @@;
    datalines;
esp w cap d cap w kon w ice w kon d esp d kon w ice d esp d
cap w esp d cap d Kon d . d kon w esp d cap w ice w kon w
kon w kon w ice d esp d kon w esp d esp w kon w cap w kon w
;
""", method="listonly")
sas.sasdata('orders').head()

,Coffee,Window
0,esp,w
1,cap,d
2,cap,w
3,kon,w
4,ice,w


In [3]:
sas.submitLST(f"""
data bmi_groups;
    set sashelp.heart;
    BMI = weight / (height**2) * 703;
    
    if BMI < 18.5 then bmi_range = "Underweight";
    else if BMI < 25 and BMI >= 18.5 then bmi_range = "Normal";
    else if BMI < 30 then bmi_range = "Overweight";
    else bmi_range = "Obese";
              
    if bmi_range = "Underweight" then bmi_range_n = "1. Underweight";
    else if bmi_range = "Normal" then bmi_range_n = "2. Normal";
    else if bmi_range = "Overweight" then bmi_range_n = "3. Overweight";
    else if bmi_range = "Obese" then bmi_range_n = "4. Obese";
    else bmi_range_n = "5. Other";
              
    keep weight height bmi bmi_range bmi_range_n;
run;

""" , method="listonly")
sas.sasdata('bmi_groups').head()


,Height,Weight,BMI,bmi_range,bmi_range_n
0,62.50,140.0,25.195520,Overweight,3. Overweight
1,59.75,194.0,38.201572,Obese,4. Obese
2,62.25,132.0,23.946969,Normal,2. Normal
3,65.75,158.0,25.693360,Overweight,3. Overweight
4,66.00,156.0,25.176309,Overweight,3. Overweight


In [4]:
sas.submitLST(f"""
    proc logistic data=sashelp.cars order=freq noprint;
        model origin (event='Asia') = mpg_city mpg_highway weight / link=glogit;
        ouput out=cars_est predprobs=INDIVIDUAL;
    
    proc print data=cars_est (obs=5);
        var origin mpg_city mpg_highway weight _from_ _into_;
    run;
""")

Obs,Origin,MPG_City,MPG_Highway,Weight,_FROM_,_INTO_
1,Asia,17,23,4451,Asia,USA
2,Asia,24,31,2778,Asia,Asia
3,Asia,22,29,3230,Asia,Asia
4,Asia,20,28,3575,Asia,USA
5,Asia,18,24,3880,Asia,USA


### proc freq 


In [5]:

sas.submitLST(f"""
proc freq data=bmi_groups;
    tables bmi_range_n;
    title "Frequency Distribution of BMI Ranges 2";
run;
title;
""")


bmi_range_n,Frequency,Percent,CumulativeFrequency,CumulativePercent
1. Underweight,89,1.71,89,1.71
2. Normal,2456,47.15,2545,48.86
3. Overweight,1970,37.82,4515,86.68
4. Obese,694,13.32,5209,100.00


In [6]:
# tables interaction of 2 vars
sas.submitLST(f"""
PROC FREQ DATA = orders; 
    TABLES Window * Coffee; 
RUN;
""")

In [7]:
# statistical options to limit outputs
sas.submitLST(f"""
PROC FREQ DATA = cars_est; 
    TABLES _from_ * _into_ / missing nocum norow nocol nofreq;
RUN;
""")

In [8]:
# statistical options to have new outputs
sas.submitLST(f"""
* Print tables for Window and Window by Coffee;
PROC FREQ DATA = orders;
    TABLES Window * Coffee / MISSING norow nocol nopercent; 
RUN;
""")

In [9]:
# long format
sas.submitLST(f"""
PROC FREQ DATA = orders noprint;
    TABLES Window * Coffee / LIST MISSING out = freqout; 
RUN;
""", method="listonly")
df = sas.sasdata('freqout').to_df()
df

,Window,Coffee,COUNT,PERCENT
0,d,NaN,1.0,3.333333
1,d,Kon,1.0,3.333333
2,d,cap,2.0,6.666667
3,d,esp,6.0,20.000000
4,d,ice,2.0,6.666667
5,d,kon,1.0,3.333333
6,w,cap,4.0,13.333333
7,w,esp,2.0,6.666667
8,w,ice,2.0,6.666667
9,w,kon,9.0,30.000000


In [10]:
df.pivot(index='Window', columns='Coffee', values='PERCENT')

Coffee,NaN,Kon,cap,esp,ice,kon
Window,,,,,,
d,3.333333,3.333333,6.666667,20.000000,6.666667,3.333333
w,NaN,NaN,13.333333,6.666667,6.666667,30.000000
